In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pickle
import meteomatics.api as api
import datetime as dt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display, Markdown, clear_output
from ipywidgets import Button, HBox, VBox
from IPython.display import Image

In [3]:

#style = {'font_size':30}

payload = widgets.IntSlider(description = 'payload (g)', min=0, max=750, value=250, step=25)
distance = widgets.IntSlider(description = 'distance (m)', min=100, max=1000, value=150, step=100)
direction = widgets.IntSlider(description = 'direction (deg)', min=0, max=360, value=30, step=10)
speed = widgets.IntSlider(description = 'speed (m/s)', min=4, max=10, value=4, step=1)
altitude = widgets.IntSlider(description = 'altitude (m)', min=25, max=100, value=50, step=5)

def on_change(v):
    x = v['new']

payload.observe(on_change, names='value')
distance.observe(on_change, names='value')
direction.observe(on_change, names='value')
speed.observe(on_change, names='value')
altitude.observe(on_change, names='value')

lat = widgets.BoundedFloatText(
    value=45.42,
    min=-90,
    max=+90,
    step=0.01,
    description='Latitude:',
    disabled=False
)

lon = widgets.BoundedFloatText(
    value=75.70,
    min=-180,
    max=+180,
    step=0.01,
    description='Longitude:',
    disabled=False
)

lat.observe(on_change, names='value')
lon.observe(on_change, names='value')

button = widgets.Button(description='Calculate')
out = widgets.Output()
def on_button_clicked(_):
      # "linking function with output"
      with out:
        clear_output()
        params = [payload.value, distance.value, direction.value, speed.value, altitude.value,lat.value, lon.value]
        battery_usage, wind_speed, wind_direction = aeolus(params)
        print(f'Predicted Battery Usage = {round(battery_usage,2)} Wh')
        print(f'Current Wind Speed at Flight Altitude = {wind_speed} m/s')
        print(f'Current Wind Direction at Flight Altitude = {wind_direction} deg (CW relative to North)')
        #print(params[5])
# linking button and function together using a button's method
button.on_click(on_button_clicked)
# displaying button and its output together


def aeolus(params):
    payload = params[0]
    distance = params[1]
    direction = params[2]
    speed = params[3]
    altitude = params[4]
    lat = params[5]
    lon = params[6]
    
    username = 'lighthousedatasciencelab_shakouri'
    password = 'wJg9Fef4uLNP6'
    
    
    coordinates= [(lat,lon)]
    
    now = dt.datetime.utcnow().replace(hour=0, minute=0, second=0, microsecond=0)
    startdate= now
    enddate= startdate #+ dt.timedelta(days=1)
    interval = dt.timedelta(hours=12)
    parameters = [f'wind_speed_{altitude}m:ms',f'wind_dir_{altitude}m:d']
    wind_query = api.query_time_series(coordinates, startdate, enddate, interval,
                                  parameters, username, password)
    wind_speed = wind_query.values.tolist()[0][0]
    wind_dir = wind_query.values.tolist()[0][1]
    rel_direction = 180 - abs(direction - wind_dir)*np.pi/180
    
    tailwind = wind_speed * np.cos(rel_direction)
    sidewind = wind_speed * np.sin(rel_direction)
    
    duration_cruise = distance/speed
    duration_takeoff = altitude//1.8761485668470712
    duration_land = altitude/1.272185544117594
    
    
    data = {'flight_phase':['take_off','cruise','land'], 'payload':[payload,payload,payload],
        'duration':[duration_takeoff,duration_cruise,duration_land], 'speed':[0,speed,0],
        'altitude_change':[altitude,0,-altitude], 'tailwind':[tailwind/2,tailwind,tailwind/2],
        'sidewind':[sidewind/2,sidewind,sidewind/2]}
    
    flight_plan = pd.DataFrame(data)
    #print(flight_plan)
    
    with open('./Aeolus_Predictor.pickle', 'rb') as f:
        predictor_model = pickle.load(f)
    
    power_consumption = predictor_model.predict(flight_plan)
    duration = flight_plan['duration']
    battery_usage = power_consumption.dot(duration)/3600
    return battery_usage, wind_speed, -wind_dir



items1 = [payload,distance,direction,speed,altitude]
#items2 =[lat,long]
left_box = VBox([items1[0], items1[1], items1[2], items1[3], items1[4]])
#middle_box = VBox([items2[0], items2[1]])
right_box = VBox([lat,lon,button,out])
HBox([left_box, right_box])


